In [29]:
import keras
import numpy as np
import matplotlib.pyplot as plt

In [30]:
import keras.datasets.mnist as mnist
(train_image, train_label), (test_image, test_label) = mnist.load_data() 

In [31]:
train_image = np.expand_dims(train_image, axis = -1)
test_image = np.expand_dims(test_image, axis = -1)

# 原始网络搭建 + 训练 + 形式2的“层结构”保存 + 形式3“参数”保存

In [32]:
model = keras.Sequential()

In [33]:
from keras import layers

In [34]:
# 第一层要给输入数据的形状：只要管最后3个维度，前面的batch维不用管
model.add( layers.Conv2D( filters=64, kernel_size=(3,3), activation = 'relu', input_shape=(28,28,1) ) )  # 其他一般都用默认
model.add( layers.Conv2D( filters=64, kernel_size=(3,3), activation='relu') )
model.add( layers.MaxPooling2D()  )  # 池化层一般都用默认的

# 进入全连接层：
model.add( layers.Flatten() )  # 把(12,12,64)全部展平为12*12*64 = 9216 —— 前面已经说过这个三维数据里都是特征！！！
model.add(layers.Dense(256, activation='relu'))
model.add( layers.Dropout(0.5) )  # 网络容量还是有些大，dropout一下
model.add(layers.Dense(10, activation='softmax'))  # 最后是10分类输出，激活用softmax多分类

In [35]:
model.compile( optimizer='adam',
               loss = 'sparse_categorical_crossentropy',  # 顺序编码
               metrics=['acc']
)

In [36]:
model.fit(train_image, train_label, epochs = 3, batch_size = 512, validation_data=(test_image, test_label) )

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 96s 2ms/step - loss: 1.1123 - acc: 0.8966 - val_loss: 0.0634 - val_acc: 0.9796
Epoch 2/3
60000/60000 [==============================] - 96s 2ms/step - loss: 0.0837 - acc: 0.9746 - val_loss: 0.0500 - val_acc: 0.9843
Epoch 3/3
60000/60000 [==============================] - 93s 2ms/step - loss: 0.0602 - acc: 0.9817 - val_loss: 0.0397 - val_acc: 0.9871


### 对原模型的“层结构”保存：

In [37]:
# 保存：
my_model_json_24 = model.to_json()

# 写入文件：
with open('my_model_json_24.json', 'w') as f:
    f.write(my_model_json_24)

### 对原模型的“层结构”加载：

In [38]:
# 加载：
from keras.models import model_from_json

# 读入
with open('my_model_json_24.json', 'r') as f:
    my_model_json_24 = f.read()
    
# 开始加载：
model1 = model_from_json(my_model_json_24)

In [39]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2359552   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)               

In [40]:
# 还是要先编译一下：因为它毕竟只停留到model.compile()之前！
# 注意：只要模型编译了，它所有的权重就会被赋上一个随机初始值！（训练前的初始值）
model1.compile( optimizer='adam',
               loss = 'sparse_categorical_crossentropy',  # 顺序编码
               metrics=['acc']
)

In [41]:
# 查看是否具有训练好的权重：
model1.evaluate(test_image, test_label)

10000/10000 [==============================] - 4s 388us/step


[37.0588640625, 0.066600002348423]

### 对原模型的“参数”保存：

In [42]:
# 保存：
model.save_weights('my_model_weights.h5')

### 用“层结构model1”来加载原模型的“参数”：

In [43]:
# model1本来还是要先编译一下，但之前已经编译了，这里可以直接加载权重；
# 用“层结构model1”来加载原模型的“参数”:
model1.load_weights('my_model_weights.h5')

In [44]:
# 测试：层结构 + 权重参数 = 功能完整的原模型？
model1.evaluate(test_image, test_label)

10000/10000 [==============================] - 4s 389us/step


[0.03968261045569307, 0.9871000051498413]

此时很明显训练好的参数已经加载到层模型上！现在的model1和原始的模型model完全一样！

说明：
- 这里用的是保存的“层模型model1”；这里也完全可以用再自定义“大小匹配”的新层模型来用 —— 很好理解。
- 目前的操作是对“**所有参数的加载**”，keras还可以进行“**部分层参数的加载**”，此时需要“**对每一层命名**”！

对“**部分层参数的加载**”的操作，见文件25